In [74]:
%load_ext autoreload
%autoreload 2
%cd C:\MAD4AG
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
C:\MAD4AG


In [75]:
import pandas as pd
import numpy as np
# from google.colab import drive
from tqdm import tqdm
from datetime import datetime
import matplotlib.pyplot as plt

In [76]:
dataf_mddr1116 = pd.read_csv('dbs/travel_survey_se_deso/se_rvu_distribution_mdre1116_deso.csv', dtype = {'d_a_kl' : 'Int32', 'd_b_kl' : 'Int32', 'd_a_pkt' : 'Int32', 'd_a_sve': 'Int32', 'd_b_sve': 'Int32','h_a_lan': 'Int32','h_b_lan': 'Int32'}, delimiter=',',  encoding='latin-1')


print('The number of people= ', len(dataf_mddr1116.uenr.unique()))

C:\Users\aglar\AppData\Local\anaconda3\envs\pycharm\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (181,370,379) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


The number of people=  38258


In [77]:
variables_dict = {"uenr":"sub_id", "bost_lan":"home_county",
                  'bost_komk':'home_municipality',
                   "arb_komk":'workplace_municipality',
                  "arb_fard":"commute_mode","arb_km":"commute_distance", "arb_lan":"workplace_county","arb_tid":"commute_time",
                  "i_datum":"date", "uedag":"day_of_week", "h_nummer":"trip_id", "d_nummer":"trip_main_id", "r_nummer":"trip_seg_id",
                  "d_ford":"trip_main_mode", "d_km":"distance_main", "d_restid":"tt_main", "d_areall":"trip_purpose_main",
                  "d_a_sve":"origin_main_domestic","d_b_sve":"desti_main_domestic",
                  'd_a_kl':"origin_main_time", 'd_b_kl': "desti_main_time",
                  "r_ford":"trip_seg_mode", "r_km":"distance_seg", # , "r_restid":"tt_seg"
                  "h_a_kl":"origin_time", "h_a_lan":"origin_county", "h_b_kl":"desti_time",
                  "h_b_lan":"desti_county", "h_km":"distance", "h_restid":"tt",
                  "h_ford":"mode", "h_are":"trip_purpose",
                  "vikt":"trip_weight", "vikt_k":"sub_weight",
                  'd_a_deso':"origin_main_deso", 'd_b_deso':"desti_main_deso",
                  'd_a_komk':"origin_main_municipality", 'd_b_komk':"desti_main_municipality",
                  'h_a_deso':"origin_deso", 'h_b_deso':"desti_deso",
                  'arb_deso':'workplace_deso', 'bost_deso':'home_deso', 'd_a_pkt':'origin_act_typ','d_b_pkt':'desti_act_typ'}

In [78]:
dataf_mddr1116= dataf_mddr1116.rename(columns=variables_dict)
dataf_mddr1116 =  dataf_mddr1116.sort_values(['sub_id', 'trip_id','trip_main_id', 'trip_seg_id'])

In [79]:
#select individuals based on days of a week (consider only weekdays)

dataf_mddr1116 = dataf_mddr1116[dataf_mddr1116.day_of_week.isin([1,2,3,4,5])]

print('The number of people having a weekday activities= ', len(dataf_mddr1116.sub_id.unique()))

The number of people having a weekday activities=  28272


In [80]:
selected_columns = list(variables_dict.values())
selected_columns = ['sub_id', 'age',
 'trip_id',
 'trip_main_id',
 'origin_main_time',
 'desti_main_time',
 'origin_act_typ',
 'desti_act_typ',
 'origin_main_domestic',
 'desti_main_domestic',
 'origin_main_deso',
 'desti_main_deso',
 'origin_county',
 'desti_county',
 'home_county',
 'home_municipality',
 'trip_weight',
 'sub_weight'
 ]

In [81]:
dataf_mddr1116= dataf_mddr1116[selected_columns]

In [82]:
# drop people less than 18 years old, keep only adults

dataf_mddr1116= dataf_mddr1116[dataf_mddr1116.age>17]


In [83]:
# drop trip legs, keep only main trips

dataf_mddr1116 = dataf_mddr1116.drop_duplicates(['sub_id','trip_id','trip_main_id'])

In [84]:
# delete individuals for whom origin_main_time or desti_main_time data is missing

invalid_origin_main_time = dataf_mddr1116[dataf_mddr1116['origin_main_time'].isnull()].sub_id.unique().tolist()
invalid_desti_main_time = dataf_mddr1116[dataf_mddr1116['desti_main_time'].isnull()].sub_id.unique().tolist()


# delete individuals having trips abroad

invalid_origin_main_domestic = dataf_mddr1116[dataf_mddr1116.origin_main_domestic==2].sub_id.unique().tolist()
invalid_desti_main_domestic = dataf_mddr1116[dataf_mddr1116.desti_main_domestic==2].sub_id.unique().tolist()

invalid_indiv = set().union(invalid_origin_main_time , invalid_desti_main_time, invalid_origin_main_domestic, invalid_desti_main_domestic)
invalid_indiv= list(invalid_indiv)

print('The number of people whose time column is missing or who have activities abroad = ',len(invalid_indiv))


dataf_mddr1116 = dataf_mddr1116[~dataf_mddr1116['sub_id'].isin(invalid_indiv) ]

The number of people whose time column is missing or who have activities abroad =  1456


In [85]:
# calculate number of activities for each individual

num_act = dataf_mddr1116.sub_id.value_counts()
num_act = num_act.reset_index()
num_act = num_act.rename(columns={"index": "sub_id", "sub_id": "act_num"})

dataf_mddr1116 = pd.merge( dataf_mddr1116, num_act, how='left', left_on = 'sub_id' , right_on = 'sub_id'  )

dataf_mddr1116['act_seq'] = dataf_mddr1116.groupby(['sub_id']).cumcount()

In [86]:
# repeat the first row in each individual to convert activity table

df_act = pd.concat([dataf_mddr1116, dataf_mddr1116.groupby('sub_id', as_index=False).first()]).sort_values(['sub_id', 'act_seq'], ignore_index=True)


df_act['act_seq'] = df_act['act_seq'] + 1
df_act['act_seq'] = df_act.groupby(['sub_id']).cumcount()
df_act['D_B_KL'] = df_act['origin_main_time'].shift(-1)

df_act.rename(columns={'desti_main_time':'D_A_KL'}, inplace=True)

In [87]:
# write 0 to the first row in each individual's activity start time
# write 2359 to the last row in each individual's activity end time

df_act['D_A_KL'][df_act['act_seq'] == 0] = 0
df_act['D_B_KL'][df_act['act_seq'] == df_act['act_num']] = 2359

In [88]:
def time_padding(x):
    if len(x) == 4:
        return x
    if len(x) == 3:
        return '0' + x
    if len(x) == 2:
        return '00' + x
    if len(x) == 1:
        return '000' + x

In [89]:
df_act.loc[:, 'D_A_KL'] = df_act['D_A_KL'].astype(str).apply(
    lambda x: time_padding(x))
df_act.loc[:, 'D_B_KL'] = df_act['D_B_KL'].astype(str).apply(
    lambda x: time_padding(x))

df_act.loc[:, 'D_A_KL'] = df_act['D_A_KL'].apply(lambda x: datetime.strptime(str(x),'%H%M'))
df_act.loc[:, 'D_B_KL'] = df_act['D_B_KL'].apply(lambda x: datetime.strptime(str(x),'%H%M'))

df_act.rename(columns={'D_A_KL':'act_start','D_B_KL':'act_end'}, inplace=True)

In [90]:
# calculate each activities duration
df_act['dur'] = (df_act['act_end'] - df_act['act_start']).dt.total_seconds() / 60  # Convert to hours

# Check for negative values in the "dur" column
negative_dur = df_act[df_act['dur'] < 0]

df_act = df_act.drop(df_act[df_act['sub_id'].isin(negative_dur['sub_id'].unique())].index)
print('The number of individual after removing individuals having negative activity duration ', len(df_act.sub_id.unique()))

The number of individual after removing individuals having negative activity duration  22323


In [91]:
# activities' start and end times in minute

df_act['h_s'] = (df_act['act_start'].dt.hour)*60 +(df_act['act_start'].dt.minute)

df_act['h_e'] = (df_act['act_end'].dt.hour)*60 +(df_act['act_end'].dt.minute)

In [92]:
df_act.desti_act_typ = np.where(df_act['act_seq'] == 0, df_act.origin_act_typ, df_act.desti_act_typ)

df_act.desti_main_deso = np.where(df_act['act_seq'] == 0, df_act.origin_main_deso, df_act.desti_main_deso)

df_act.desti_county = np.where(df_act['act_seq'] == 0, df_act.origin_county, df_act.desti_county)

df_act.rename(columns={'desti_act_typ': 'purpose', 'desti_main_deso': 'act_deso', 'desti_county': 'county'}, inplace=True)


In [93]:
df_act = df_act[['sub_id', 'act_num', 'act_seq', 'act_start', 'act_end', 'dur', 'h_s', 'h_e',
       'purpose', 'act_deso', 'county', 'home_county', 'home_municipality', 'trip_weight', 'sub_weight']]

Start and end locations are restricted as you see below in the survey. Other places are numbered with 10 and it includes shopping places,  restaurants,  entertainment places, and etc.

   1 = 'Rp's place of residence, population registration address'
   2 = 'Other permanent residence, Rp's (for military units also regiment)'
   3 = 'RP: S (MAIN) LEISURE RESIDENCE WITH ADDRESS'
   4 = 'RP: S (MAIN) WORKPLACE (ONLY WORKPLACE IN SWEDEN) with address'
   5 = 'Rp's workplace 2, 3 etc, workplace abroad'
   6 = 'RP: S (MAIN) SCHOOL WITH ADDRESS'
   7 = 'Temporary accommodation'
   8 = 'Rp's school 2,3 etc, another school, internship etc.
   9 = 'Rp's holiday home, also rented (where we have not received the address before)'
   10 = 'Other place'
   88 = 'Don't know'
   99 = 'Don't want to reply'

In [94]:
# drop irregular activity types

irregular_act = df_act[df_act['purpose'].isin([7, 88, 99])]

df_act = df_act.drop(df_act[df_act['sub_id'].isin(irregular_act['sub_id'].unique())].index)

len(df_act.sub_id.unique())

21061

In [95]:
df_act['purpose'].replace([2, 3, 9], 1, inplace=True)

df_act['purpose'].replace([8], 6, inplace=True)

df_act['purpose'].replace([5], 4, inplace=True)


mapping = {1: 'Home', 4: 'Work', 6: 'School', 10: 'Other'}
df_act['purpose'] = df_act['purpose'].map(mapping)



In [96]:
# remove ppl not having a home activity at their daily plan

ppl_having_home = df_act[(df_act.purpose=='Home')].sub_id.unique()

df_act = df_act[df_act.sub_id.isin(ppl_having_home)]

In [97]:
#remove ppl visiting different home locations in a day

numberof_home = df_act[df_act.purpose=='Home'].groupby(['sub_id','purpose'])['act_deso'].nunique().reset_index()

df_act = df_act[~df_act.sub_id.isin(numberof_home['sub_id'][numberof_home.act_deso>1])]

In [28]:
# save the activity table

df_act.to_pickle(f'./dbs/survey_act_weekday.pkl')